# D. PERFORMANCE UNDER NEW WORK CONDITIONS
In this experiment, we will evaluate effect of the proposed few-shot learning method to address the fourth challenge in limited data fault diagnosis. We mainly pay attention in unbalanced working condition. When new categories appear, we would like to evaluate how well the few-shot performance. We use domain adaptation to simulate new work conditions.

# 新工作条件下的性能
在本实验中，我们将评估所提出的少样本学习方法在解决有限数据故障诊断中的第四个挑战的效果。 我们主要关注不平衡的工作条件。 当出现新的类别时，我们想评估few-shot 的表现如何。 我们使用领域适应来模拟新的工作条件。

In [1]:
%matplotlib inline
import tensorflow as tf
import tensorflow
# set the memory usage
from tensorflow.python.keras.backend import set_session

tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=tf_config))

import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
import imp
import pandas as pd
import cwru 
import keras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

## Load models

In [2]:
import models
# imp.reload(models)
window_size = 2048

siamese_net = models.load_siamese_net_my_mew((window_size,2))
print('\nsiamese_net summary:')
siamese_net.summary()


siamese_net summary:
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 2048, 2)]    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2048, 2)]    0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 100)          137763      input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           model

In [3]:
print('\nsequential_3 is WDCNN:')
siamese_net.layers[2].summary()


sequential_3 is WDCNN:
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048, 2)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 128, 16)      2064        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 16)      64          conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 16)      0           batch_normalization[0][0]        
______________________________________________________________________

In [4]:
wdcnn_net = models.load_wdcnn_net()
print('\nwdcnn_net summary:')
wdcnn_net.summary()

52806

wdcnn_net summary:
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 2048, 2)]         0         
_________________________________________________________________
sequential (Sequential)      (None, 100)               51796     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 52,806
Trainable params: 52,806
Non-trainable params: 0
_________________________________________________________________


## Config

In [5]:
import keras
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint,EarlyStopping

import siamese
# imp.reload(siamese)
import utils
# imp.reload(utils)

settings = {
  "N_way": 10,           # how many classes for testing one-shot tasks>
  "batch_size": 32,
  "best": -1,
  "evaluate_every": 600,   # interval for evaluating on one-shot tasks
  "loss_every": 20,      # interval for printing loss (iterations)
  "n_iter": 15000,
  "n_val": 2,          #how many one-shot tasks to validate on?
  "n": 0,
  "save_path":"",
  "save_weights_file": "weights-best-10-oneshot-low-data.hdf5"
}
loads ={'A':'1772', 'B':'1750', 'C':'1730'}

times = 10

In [6]:
def WC_train_and_test(exp_name,exps,hps_transfer,is_training):
    window_size = 2048
    for (hp_from,hp_to) in hps_transfer:
        data = cwru.CWRU(['12DriveEndFault'], [loads[i] for i in hp_from],window_size)
        print(len(data.X_train),len(data.X_test))
        data_test = cwru.CWRU(['12DriveEndFault'], [loads[i] for i in hp_to],window_size)
        print(len(data_test.X_train),len(data_test.X_test))
        data.X_test, data.y_test = data_test.X_test, data_test.y_test

        scores_hps = []
        train_classes = sorted(list(set(data.y_train)))
        train_indices = [np.where(data.y_train == i)[0] for i in train_classes]
        for exp in exps:
            # enable the random seed for experimental reproduction
            np.random.seed(exp)
            scores_1_shot = []
            scores_5_shot = []
            scores_wdcnn = []
            num = int(exp/len(train_classes))
            settings['evaluate_every'] = 300 if exp<1000 else 600
            print(settings['evaluate_every'])
            for time_idx in range(times):
                np.random.seed(0)
                print("\n样本数%s-第%s次训练"%(exp,time_idx) + '*'*80)
                settings["save_path"] = "tmp/%s/size_%s/%s2%s/time_%s/" % (exp_name,exp,''.join(hp_from),
                                                                           '_'.join(hp_to),time_idx)
                data._mkdir(settings["save_path"])

                train_idxs = []
                val_idxs = []
                for i, c in enumerate(train_classes):
                    select_idx = train_indices[i][np.random.choice(len(train_indices[i]), num, replace=False)]
                    split = int(0.6*num)
                    train_idxs.extend(select_idx[:split])
                    end = num
                    if(0.4*num>100):
                        end = split+100
                    val_idxs.extend(select_idx[split:end])
                X_train, y_train = data.X_train[train_idxs],data.y_train[train_idxs], 
                X_val, y_val = data.X_train[val_idxs],data.y_train[val_idxs], 

                print(train_idxs[0:10])
                print(val_idxs[0:10])
            
                # load one-shot model and training
                siamese_net = models.load_siamese_net_my_mew()
                siamese_loader = siamese.Siamese_Loader(X_train,
                                                    y_train,
                                                    X_val,
                                                    y_val)
                if(is_training):
                    print(siamese.train_and_test_oneshot(settings,siamese_net,siamese_loader))

                # test 1_shot and 5_shot
                print("load best weights",settings["save_path"] + settings['save_weights_file'])
                siamese_net.load_weights(settings["save_path"] + settings['save_weights_file'])
                siamese_loader.set_val(data.X_test,data.y_test)
                s = 'val'
                preds_5_shot = []
                scores = []
                for k in range(5):
                    val_acc, preds, prods = siamese_loader.test_oneshot2(siamese_net,len(siamese_loader.classes[s]),
                                                                 len(siamese_loader.data[s]),verbose=False)
        #             confusion_plot(preds[:,1],preds[:,0])
                    print(val_acc)
                    scores.append(val_acc)
                    preds_5_shot.append(preds[:,1])
                preds = []
                for line in np.array(preds_5_shot).T:
                    preds.append(np.argmax(np.bincount(line)))
        #         confusion_plot(np.array(preds),data.y_test) 
                score_5_shot = accuracy_score(data.y_test,np.array(preds))*100
                print('5_shot:',score_5_shot)
                scores_1_shot.append(scores[0])
                scores_5_shot.append(score_5_shot)


                # load wdcnn model and training
                y_train = keras.utils.to_categorical(y_train, data.nclasses)
                y_val = keras.utils.to_categorical(y_val, data.nclasses)
                y_test = keras.utils.to_categorical(data.y_test, data.nclasses)

                earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')
                # checkpoint
                filepath="%s/weights-best-10-cnn-low-data.hdf5" % (settings["save_path"])
                checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
                tb = keras.callbacks.TensorBoard(log_dir='./tmp/logs', histogram_freq=4, 
                                                 batch_size=32, write_graph=True, 
                                                 write_grads=False, write_images=True, 
                                                 embeddings_freq=0, embeddings_layer_names=None, 
                                                 embeddings_metadata=None, embeddings_data=None, 
                                                 update_freq='epoch')

                # callbacks_list = [earlyStopping,checkpoint,tb]
                callbacks_list = [earlyStopping,checkpoint]

                wdcnn_net = models.load_wdcnn_net()

                if(is_training):
                    wdcnn_net.fit(X_train, y_train,
                              batch_size=32,
                              epochs=300,
                              verbose=0,
                              callbacks=callbacks_list,
                              validation_data=(X_val, y_val))

                # test wdcnn
                wdcnn_net.load_weights(filepath)
                score = wdcnn_net.evaluate(data.X_test, y_test, verbose=0)
                print('Test loss:', score[0])
                print('Test accuracy:', score[1])
                scores_wdcnn.append(score[1]*100)


            a =pd.DataFrame(np.array(scores_1_shot))
    #         a.columns = str(exp)
            a.to_csv("tmp/%s/size_%s/%s2%s/scores_1_shot.csv" % (exp_name,exp,''.join(hp_from),'_'.join(hp_to)),index=True)


            a =pd.DataFrame(np.array(scores_5_shot))
    #         a.columns = str(exp)
            a.to_csv("tmp/%s/size_%s/%s2%s/scores_5_shot.csv" % (exp_name,exp,''.join(hp_from),'_'.join(hp_to)),index=True)


            a =pd.DataFrame(np.array(scores_wdcnn))
    #         a.columns = str(exp)
            a.to_csv("tmp/%s/size_%s/%s2%s/scores_wdcnn.csv" % (exp_name,exp,''.join(hp_from),'_'.join(hp_to)),index=True)  

        
def WC_analysis(exp_name,exps,hps_transfer):
    scores_1_shot_all = pd.DataFrame()
    scores_5_shot_all = pd.DataFrame()
    scores_wdcnn_all = pd.DataFrame()
    for (hp_from,hp_to) in hps_transfer:
        for exp in exps:
            hps = "%s2%s" % (''.join(hp_from),'_'.join(hp_to))
            file_path = "tmp/%s/size_%s/%s" % (exp_name,exp,hps)
            tmp_data = pd.read_csv("%s/scores_1_shot.csv" % (file_path), 
                                   sep=',', index_col=0)
            tmp_data['hps'] = hps
            tmp_data['exp'] = exp 
            scores_1_shot_all = pd.concat([scores_1_shot_all,tmp_data],axis=0)

            tmp_data = pd.read_csv("%s/scores_5_shot.csv" % (file_path), 
                                   sep=',', index_col=0)
            tmp_data['hps'] = hps
            tmp_data['exp'] = exp 
            scores_5_shot_all = pd.concat([scores_5_shot_all,tmp_data],axis=0)

            tmp_data = pd.read_csv("%s/scores_wdcnn.csv" % (file_path), 
                                   sep=',', index_col=0)
            tmp_data['hps'] = hps
            tmp_data['exp'] = exp 
            scores_wdcnn_all = pd.concat([scores_wdcnn_all,tmp_data],axis=0)


    scores_1_shot_all.to_csv("tmp/%s/scores_1_shot_all.csv" % (exp_name), float_format='%.6f', index=True)
    scores_5_shot_all.to_csv("tmp/%s/scores_5_shot_all.csv" % (exp_name), float_format='%.6f', index=True)
    scores_wdcnn_all.to_csv("tmp/%s/scores_wdcnn_all.csv" % (exp_name), float_format='%.6f', index=True)

    scores_1_shot_all['model'] = 'One-shot'
    scores_5_shot_all['model'] = 'Five-shot'
    scores_wdcnn_all['model'] = 'WDCNN'

    scores_all = pd.concat([scores_1_shot_all,scores_5_shot_all,scores_wdcnn_all],axis=0)
    scores_all.to_csv("tmp/%s/scores_all.csv" % (exp_name), float_format='%.6f', index=True)   
    
    return scores_all

## Scenario setting for domain adaption

In [7]:
exp_name = "EXP-D"
exps = [1500]
hps_transfer = [
#     (['A'],['B']),
#     (['A'],['C']),
#     (['B'],['A']),
#     (['B'],['C']),
    (['C'],['A']),
    (['C'],['B']),
]

# is_training = False    # enable or disable train models. if enable training, save best models will be update.
is_training = True # 如果是想重新训练模型的话，设置为True，不然就是从对应的文件夹中来下载对应的训练好的模型权重

### Training & Testing

In [8]:
WC_train_and_test(exp_name,exps,hps_transfer,is_training)

Datasets/CWRU 12DriveEndFault 1730
idx： 0
fpath： Datasets/CWRU\NormalBaseline\1730\Normal.mat
time_series.shape: (485643, 2)
idx： 1
fpath： Datasets/CWRU\12DriveEndFault\1730\0.007-Ball.mat
time_series.shape: (121556, 2)
idx： 2
fpath： Datasets/CWRU\12DriveEndFault\1730\0.014-Ball.mat
time_series.shape: (122136, 2)
idx： 3
fpath： Datasets/CWRU\12DriveEndFault\1730\0.021-Ball.mat
time_series.shape: (122136, 2)
idx： 4
fpath： Datasets/CWRU\12DriveEndFault\1730\0.007-InnerRace.mat
time_series.shape: (122917, 2)
idx： 5
fpath： Datasets/CWRU\12DriveEndFault\1730\0.014-InnerRace.mat
time_series.shape: (121701, 2)
idx： 6
fpath： Datasets/CWRU\12DriveEndFault\1730\0.021-InnerRace.mat
time_series.shape: (121991, 2)
idx： 7
fpath： Datasets/CWRU\12DriveEndFault\1730\0.007-OuterRace6.mat
time_series.shape: (122571, 2)
idx： 8
fpath： Datasets/CWRU\12DriveEndFault\1730\0.014-OuterRace6.mat
time_series.shape: (121991, 2)
idx： 9
fpath： Datasets/CWRU\12DriveEndFault\1730\0.021-OuterRace6.mat
time_series.shape:

Test loss: 0.7552144527435303
Test accuracy: 0.800000011920929

样本数1500-第1次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6197, 4815, 3463, 5147, 2666, 1973, 2405, 624, 5116, 1172]
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X_val的shape (600, 2048, 2)
Siamese_Loader的y_val的shape (600,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 600, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_1500/C2A/time_1/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 70.0
580 轮: 损失值：0.00279, 正确率:1.00000，验证集正确率100.00000:
iteration 600 evaluating: 100.0
1780 轮: 损失值：0.00031, 正确率:1.00000，验证集正确率100.00000:
iteration 1800 evaluating: 100.0
2980 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 3000 evaluating: 100.0
4180 轮: 损失值：0.00008, 正确率:1.00000，验证集正确率100.000

Test loss: 0.7525198459625244
Test accuracy: 0.7960000038146973

样本数1500-第2次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6197, 4815, 3463, 5147, 2666, 1973, 2405, 624, 5116, 1172]
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X_val的shape (600, 2048, 2)
Siamese_Loader的y_val的shape (600,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 600, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_1500/C2A/time_2/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 78.0
580 轮: 损失值：0.00168, 正确率:1.00000，验证集正确率100.00000:
iteration 600 evaluating: 100.0
1180 轮: 损失值：0.00045, 正确率:1.00000，验证集正确率100.00000:
iteration 1200 evaluating: 100.0
1780 轮: 损失值：0.00058, 正确率:1.00000，验证集正确率100.00000:
iteration 1800 evaluating: 100.0
2380 轮: 损失值：0.00007, 正确率:1.00000，验证集正确率100.00

Test loss: 1.0602270364761353
Test accuracy: 0.7960000038146973

样本数1500-第3次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6197, 4815, 3463, 5147, 2666, 1973, 2405, 624, 5116, 1172]
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X_val的shape (600, 2048, 2)
Siamese_Loader的y_val的shape (600,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 600, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_1500/C2A/time_3/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 70.0
580 轮: 损失值：0.00172, 正确率:1.00000，验证集正确率100.00000:
iteration 600 evaluating: 100.0
1180 轮: 损失值：0.00145, 正确率:1.00000，验证集正确率100.00000:
iteration 1200 evaluating: 100.0
1780 轮: 损失值：0.00061, 正确率:1.00000，验证集正确率100.00000:
iteration 1800 evaluating: 100.0
2380 轮: 损失值：0.00020, 正确率:1.00000，验证集正确率100.00

7780 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 7800 evaluating: 100.0
8380 轮: 损失值：0.00581, 正确率:1.00000，验证集正确率100.00000:
iteration 8400 evaluating: 100.0
8980 轮: 损失值：0.00006, 正确率:1.00000，验证集正确率100.00000:
iteration 9000 evaluating: 100.0
9580 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 9600 evaluating: 100.0
10180 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 10200 evaluating: 100.0
10780 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 10800 evaluating: 100.0
11380 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 11400 evaluating: 100.0
11980 轮: 损失值：0.00009, 正确率:1.00000，验证集正确率100.00000:
iteration 12000 evaluating: 100.0
12580 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率100.00000:
iteration 12600 evaluating: 100.0
13180 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 13200 evaluating: 100.0
13780 轮: 损失值：0.00006, 正确率:1.00000，验证集正确率100.00000:
iteration 13800 evaluating: 100.0
14380 轮: 损失值：0.00008, 正确率:1.00000，验证集正确率100.00000:
iteration 14400 evalua

...

Test loss: 0.6531296372413635
Test accuracy: 0.8920000195503235

样本数1500-第7次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6197, 4815, 3463, 5147, 2666, 1973, 2405, 624, 5116, 1172]
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X_val的shape (600, 2048, 2)
Siamese_Loader的y_val的shape (600,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 600, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_1500/C2B/time_7/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 77.16666666666667
580 轮: 损失值：0.00262, 正确率:1.00000，验证集正确率100.00000:
iteration 600 evaluating: 100.0
1780 轮: 损失值：0.03430, 正确率:0.96875，验证集正确率100.00000:
iteration 1800 evaluating: 100.0
2380 轮: 损失值：0.00009, 正确率:1.00000，验证集正确率100.00000:
iteration 2400 evaluating: 100.0
2980 轮: 损失值：0.00019, 正确率:1.00000

Test loss: 0.8463761806488037
Test accuracy: 0.7519999742507935

样本数1500-第8次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6197, 4815, 3463, 5147, 2666, 1973, 2405, 624, 5116, 1172]
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X_val的shape (600, 2048, 2)
Siamese_Loader的y_val的shape (600,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 600, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_1500/C2B/time_8/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 71.0
580 轮: 损失值：0.00271, 正确率:1.00000，验证集正确率100.00000:
iteration 600 evaluating: 100.0
1180 轮: 损失值：0.02847, 正确率:0.96875，验证集正确率92.66667::
iteration 1200 evaluating: 100.0
1780 轮: 损失值：0.00803, 正确率:1.00000，验证集正确率100.00000:
iteration 1800 evaluating: 100.0
2380 轮: 损失值：0.00019, 正确率:1.00000，验证集正确率100.00

6580 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率100.00000:
iteration 6600 evaluating: 100.0
7180 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 7200 evaluating: 100.0
7780 轮: 损失值：0.00006, 正确率:1.00000，验证集正确率100.00000:
iteration 7800 evaluating: 100.0
8380 轮: 损失值：0.00014, 正确率:1.00000，验证集正确率100.00000:
iteration 8400 evaluating: 100.0
9580 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率100.00000:
iteration 9600 evaluating: 100.0
10180 轮: 损失值：0.00026, 正确率:1.00000，验证集正确率100.00000:
iteration 10200 evaluating: 100.0
10780 轮: 损失值：0.00006, 正确率:1.00000，验证集正确率100.00000:
iteration 10800 evaluating: 100.0
11380 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率100.00000:
iteration 11400 evaluating: 100.0
11980 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 12000 evaluating: 100.0
12580 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 12600 evaluating: 100.0
13180 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率100.00000:
iteration 13200 evaluating: 100.0
13780 轮: 损失值：0.00019, 正确率:1.00000，验证集正确率100.00000:
iteration 13800 evaluati

Test loss: 0.6577162146568298
Test accuracy: 0.7360000014305115

样本数1500-第9次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6197, 4815, 3463, 5147, 2666, 1973, 2405, 624, 5116, 1172]
Siamese_Loader的X_train的shape (900, 2048, 2)
Siamese_Loader的y_train的shape (900,)
Siamese_Loader的X_val的shape (600, 2048, 2)
Siamese_Loader的y_val的shape (600,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 600, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_1500/C2B/time_9/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 70.33333333333333
580 轮: 损失值：0.00162, 正确率:1.00000，验证集正确率100.00000:
iteration 600 evaluating: 100.0
1180 轮: 损失值：0.00051, 正确率:1.00000，验证集正确率100.00000:
iteration 1200 evaluating: 100.0
1780 轮: 损失值：0.00021, 正确率:1.00000，验证集正确率100.00000:
iteration 1800 evaluating: 100.0
2380 轮: 损失值：0.00061, 正确率:1.00000

8380 轮: 损失值：0.00079, 正确率:1.00000，验证集正确率100.00000:
iteration 8400 evaluating: 100.0
8980 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率100.00000:
iteration 9000 evaluating: 100.0
9580 轮: 损失值：0.00008, 正确率:1.00000，验证集正确率100.00000:
iteration 9600 evaluating: 100.0
10180 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 10200 evaluating: 100.0
10780 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率100.00000:
iteration 10800 evaluating: 100.0
11380 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 11400 evaluating: 100.0
11980 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:
iteration 12000 evaluating: 100.0
12580 轮: 损失值：0.00019, 正确率:1.00000，验证集正确率100.00000:
iteration 12600 evaluating: 100.0
13180 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率100.00000:
iteration 13200 evaluating: 100.0
14380 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率100.00000:
iteration 14400 evaluating: 100.0
14980 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率100.00000:100.0
load best weights tmp/EXP-D/size_1500/C2B/time_9/weights-best-10-oneshot-low-data.hdf5
60.0
60.0
60.

### Analysis

In [9]:
scores_all = WC_analysis(exp_name,exps,hps_transfer)
scores_all_mean = scores_all.groupby(['exp','model','hps']).mean().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model','hps']).std().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_all_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_all_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std

(                    0           
 hps               C2A        C2B
 exp  model                      
 1500 WDCNN      75.04  80.680001
      One-shot   57.72  63.320000
      Five-shot  57.72  63.320000,
                        0          
 hps                  C2A       C2B
 exp  model                        
 1500 WDCNN      5.993182  5.376659
      One-shot   4.109826  5.756697
      Five-shot  4.109826  5.756697)

## Scenario setting for A2C and C2A

In [10]:
exp_name = "EXP-D"
# 90 and 6600 exp have trained done before
exps = [60,120,200,300,600,900]
hps_transfer = [
#     (['A'],['B']),
    (['A'],['C']),
#     (['B'],['A']),
#     (['B'],['C']),
    (['C'],['A']),
#     (['C'],['B']),
]

### Training & Testing

In [ ]:
WC_train_and_test(exp_name,exps,hps_transfer,is_training)

Datasets/CWRU 12DriveEndFault 1772
idx： 0
fpath： Datasets/CWRU\NormalBaseline\1772\Normal.mat
time_series.shape: (483903, 2)
idx： 1
fpath： Datasets/CWRU\12DriveEndFault\1772\0.007-Ball.mat
time_series.shape: (121410, 2)
idx： 2
fpath： Datasets/CWRU\12DriveEndFault\1772\0.014-Ball.mat
time_series.shape: (122136, 2)
idx： 3
fpath： Datasets/CWRU\12DriveEndFault\1772\0.021-Ball.mat
time_series.shape: (121701, 2)
idx： 4
fpath： Datasets/CWRU\12DriveEndFault\1772\0.007-InnerRace.mat
time_series.shape: (121991, 2)
idx： 5
fpath： Datasets/CWRU\12DriveEndFault\1772\0.014-InnerRace.mat
time_series.shape: (121846, 2)
idx： 6
fpath： Datasets/CWRU\12DriveEndFault\1772\0.021-InnerRace.mat
time_series.shape: (121556, 2)
idx： 7
fpath： Datasets/CWRU\12DriveEndFault\1772\0.007-OuterRace6.mat
time_series.shape: (122426, 2)
idx： 8
fpath： Datasets/CWRU\12DriveEndFault\1772\0.014-OuterRace6.mat
time_series.shape: (122136, 2)
idx： 9
fpath： Datasets/CWRU\12DriveEndFault\1772\0.021-OuterRace6.mat
time_series.shape:

Test loss: 0.9174000024795532
Test accuracy: 0.7599999904632568

样本数60-第1次训练********************************************************************************
[5466, 935, 183, 1058, 1789, 5382, 2831, 789, 5770, 4271]
[5314, 3021, 1588, 4151, 6526, 1386, 145, 5271, 5861, 1587]
Siamese_Loader的X_train的shape (30, 2048, 2)
Siamese_Loader的y_train的shape (30,)
Siamese_Loader的X_val的shape (30, 2048, 2)
Siamese_Loader的y_val的shape (30,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_60/A2C/time_1/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 73.33333333333333
280 轮: 损失值：0.00090, 正确率:1.00000，验证集正确率90.00000:
iteration 300 evaluating: 90.0
580 轮: 损失值：0.00140, 正确率:1.00000，验证集正确率80.00000:
iteration 600 evaluating: 93.33333333333333
8080 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率93.33333::
iteration 8100 evaluating: 100.0
14980 轮: 损失值：0.00419, 正确率:1.00000，

Test loss: 0.9361289143562317
Test accuracy: 0.7200000286102295

样本数60-第2次训练********************************************************************************
[5466, 935, 183, 1058, 1789, 5382, 2831, 789, 5770, 4271]
[5314, 3021, 1588, 4151, 6526, 1386, 145, 5271, 5861, 1587]
Siamese_Loader的X_train的shape (30, 2048, 2)
Siamese_Loader的y_train的shape (30,)
Siamese_Loader的X_val的shape (30, 2048, 2)
Siamese_Loader的y_val的shape (30,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_60/A2C/time_2/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...



iteration 0 evaluating: 73.33333333333333
280 轮: 损失值：0.00144, 正确率:1.00000，验证集正确率90.00000:
iteration 300 evaluating: 83.33333333333333
580 轮: 损失值：0.00073, 正确率:1.00000，验证集正确率76.66667:
iteration 600 evaluating: 83.33333333333333
1180 轮: 损失值：0.00024, 正确率:1.00000，验证集正确率86.66667:
iteration 1200 evaluating: 86.66666666666667
1780 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率73.33333:
iteration 1800 evaluating: 86.66666666666667
2080 轮: 损失值：0.00013, 正确率:1.00000，验证集正确率83.33333:
iteration 2100 evaluating: 86.66666666666667
2380 轮: 损失值：0.00010, 正确率:1.00000，验证集正确率90.00000:
iteration 2400 evaluating: 86.66666666666667
2980 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率96.66667:
iteration 3000 evaluating: 86.66666666666667
3280 轮: 损失值：0.00101, 正确率:1.00000，验证集正确率90.00000:
iteration 3300 evaluating: 90.0
3580 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率96.66667:
iteration 3600 evaluating: 90.0
3880 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率93.33333:
iteration 3900 evaluating: 93.33333333333333
4180 轮: 损失值：0.00003, 正确率:1.00000，验证集正确率96.66667

Test loss: 0.6608461737632751
Test accuracy: 0.8999999761581421

样本数60-第3次训练********************************************************************************
[5466, 935, 183, 1058, 1789, 5382, 2831, 789, 5770, 4271]
[5314, 3021, 1588, 4151, 6526, 1386, 145, 5271, 5861, 1587]
Siamese_Loader的X_train的shape (30, 2048, 2)
Siamese_Loader的y_train的shape (30,)
Siamese_Loader的X_val的shape (30, 2048, 2)
Siamese_Loader的y_val的shape (30,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_60/A2C/time_3/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 70.0
280 轮: 损失值：0.00628, 正确率:1.00000，验证集正确率80.00000:
iteration 300 evaluating: 83.33333333333333
580 轮: 损失值：0.00129, 正确率:1.00000，验证集正确率73.33333:
iteration 600 evaluating: 90.0
3580 轮: 损失值：0.00004, 正确率:1.00000，验证集正确率90.00000:
iteration 3600 evaluating: 90.0
4180 轮: 损失值：0.00005, 正确率:1.00000，验证集正确率90.00000:


...

Test loss: 0.08748167008161545
Test accuracy: 0.9760000109672546

样本数200-第3次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6281, 1373, 1555, 4709, 3692, 3966, 804, 2434, 2378, 4862]
Siamese_Loader的X_train的shape (120, 2048, 2)
Siamese_Loader的y_train的shape (120,)
Siamese_Loader的X_val的shape (80, 2048, 2)
Siamese_Loader的y_val的shape (80,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_200/A2C/time_3/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 71.25
280 轮: 损失值：0.01487, 正确率:1.00000，验证集正确率95.00000:
iteration 300 evaluating: 95.0
580 轮: 损失值：0.00252, 正确率:1.00000，验证集正确率98.75000:
iteration 600 evaluating: 98.75
880 轮: 损失值：0.00243, 正确率:1.00000，验证集正确率98.75000:
iteration 900 evaluating: 98.75
1180 轮: 损失值：0.00127, 正确率:1.00000，验证集正确率98.75000:
iterat

14980 轮: 损失值：0.00002, 正确率:1.00000，验证集正确率97.50000::100.0
load best weights tmp/EXP-D/size_200/A2C/time_3/weights-best-10-oneshot-low-data.hdf5
98.4
98.4
98.4
98.4
98.4
5_shot: 98.4
52806


Test loss: 0.20878784358501434
Test accuracy: 0.9480000138282776

样本数200-第4次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6281, 1373, 1555, 4709, 3692, 3966, 804, 2434, 2378, 4862]
Siamese_Loader的X_train的shape (120, 2048, 2)
Siamese_Loader的y_train的shape (120,)
Siamese_Loader的X_val的shape (80, 2048, 2)
Siamese_Loader的y_val的shape (80,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_200/A2C/time_4/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 63.75
280 轮: 损失值：0.00662, 正确率:1.00000，验证集正确率93.75000:
iteration 300 evaluating: 95.0
580 轮: 损失值：0.00648, 正确率:1.00000，验证集正确率98.75000:
iteration 600 evaluating: 97.5
880 轮: 损失值：0.00122, 正确率:1.00000，验证集正确率98.75000:
iteration 900 evaluating: 98.75
1180 轮: 损失值：0.00334, 正确率:1.00000，验证集正确率98.75000:
iterati

Test loss: 0.2096635401248932
Test accuracy: 0.9599999785423279

样本数200-第5次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6281, 1373, 1555, 4709, 3692, 3966, 804, 2434, 2378, 4862]
Siamese_Loader的X_train的shape (120, 2048, 2)
Siamese_Loader的y_train的shape (120,)
Siamese_Loader的X_val的shape (80, 2048, 2)
Siamese_Loader的y_val的shape (80,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_200/A2C/time_5/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 68.75
280 轮: 损失值：0.00435, 正确率:1.00000，验证集正确率96.25000:
iteration 300 evaluating: 97.5
580 轮: 损失值：0.00520, 正确率:1.00000，验证集正确率97.50000:
iteration 600 evaluating: 97.5
1480 轮: 损失值：0.00127, 正确率:1.00000，验证集正确率98.75000::
iteration 1500 evaluating: 100.0
1780 轮: 损失值：0.00040, 正确率:1.00000，验证集正确率98.75000::
iter

Test loss: 0.3152170181274414
Test accuracy: 0.9679999947547913

样本数200-第6次训练********************************************************************************
[5466, 935, 183, 5314, 3021, 1588, 5957, 4922, 5507, 6239]
[6281, 1373, 1555, 4709, 3692, 3966, 804, 2434, 2378, 4862]
Siamese_Loader的X_train的shape (120, 2048, 2)
Siamese_Loader的y_train的shape (120,)
Siamese_Loader的X_val的shape (80, 2048, 2)
Siamese_Loader的y_val的shape (80,)
{'N_way': 10, 'batch_size': 32, 'best': -1, 'evaluate_every': 300, 'loss_every': 20, 'n_iter': 15000, 'n_val': 2, 'n': 0, 'save_path': 'tmp/EXP-D/size_200/A2C/time_6/', 'save_weights_file': 'weights-best-10-oneshot-low-data.hdf5'}
training...

iteration 0 evaluating: 70.0
280 轮: 损失值：0.01229, 正确率:1.00000，验证集正确率97.50000:
iteration 300 evaluating: 98.75
580 轮: 损失值：0.00558, 正确率:1.00000，验证集正确率98.75000:
iteration 600 evaluating: 98.75
880 轮: 损失值：0.00065, 正确率:1.00000，验证集正确率98.75000:
iteration 900 evaluating: 98.75
1180 轮: 损失值：0.00423, 正确率:1.00000，验证集正确率98.75000:
iterati

### Analysis

exps = [60,90,120,200,300,600,900,1500]
scores_all = WC_analysis(exp_name,exps,hps_transfer)
scores_all_mean = scores_all.groupby(['exp','model','hps']).mean().unstack().unstack().T \
            .sort_values(by=['model'],ascending=0).sort_values(by=['hps'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model','hps']).std().unstack().unstack().T \
            .sort_values(by=['model'],ascending=0).sort_values(by=['hps'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_AC_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_AC_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std

## New scenario setting for domain adaption

In [ ]:
exp_name = "EXP-D-NEW"
exps = [1500]
hps_transfer = [
    (['A','B'],['C']),
    (['A','C'],['B']),
    (['B','C'],['A']),
]

### Training & Testing

In [ ]:
WC_train_and_test(exp_name,exps,hps_transfer,is_training)

### Analysis

In [ ]:
scores_all = WC_analysis(exp_name,exps,hps_transfer)
scores_all_mean = scores_all.groupby(['exp','model','hps']).mean().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_std = scores_all.groupby(['exp','model','hps']).std().unstack()\
                .sort_values(by=['model'],ascending=0).sort_values(by=['exp'],ascending=1)
scores_all_mean.to_csv("tmp/%s/scores_all_mean.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_std.to_csv("tmp/%s/scores_all_std.csv" % (exp_name), float_format='%.2f', index=True)
scores_all_mean, scores_all_std